In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_squared_log_error,make_scorer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv("train(통합)_noECLO.csv")
test = pd.read_csv("test(통합본).csv")

In [3]:
def gu(address):
    return address.split()[0]
def dong(address):
    return address.split()[1]

In [4]:
for df in [train,test]:
    df['구'] = df['주소'].apply(gu)
    df['동'] = df['주소'].apply(dong)
    df.drop('주소',axis=1,inplace=True)
    

In [5]:
train.columns

Index(['요일', '기상상태', '도로형태', '노면상태', '사고유형', '연휴', '년도', '월', '일', '시간',
       '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3', '사망자수',
       '중상자수', '경상자수', '부상자수', '구', '동'],
      dtype='object')

In [6]:
train = train[['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동', '연휴', '년도', '월', '일', '시간',
       '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3','사망자수', '중상자수', '경상자수', '부상자수']]
test = test[['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동', '연휴', '년도', '월', '일', '시간',
       '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3']]

In [7]:
categorical_features = list(train.dtypes[train.dtypes == "object"].index)
display(categorical_features)

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train[i]) 
    train[i]=le.transform(train[i])
    
    for case in np.unique(test[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test[i]=le.transform(test[i])

['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동']

In [9]:
X = train.drop(['사망자수', '중상자수', '경상자수', '부상자수'],axis=1).values
y = train[['사망자수', '중상자수', '경상자수', '부상자수']].values

In [13]:
sc = StandardScaler()
X_standard = sc.fit_transform(X)
test_standard = sc.transform(test.iloc[:,:])

In [14]:
x_train,x_test,y_train,y_test = train_test_split(X_standard,y,test_size=0.2)

In [15]:
y

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       ...,
       [0, 2, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 2, 0]], dtype=int64)

In [16]:
from catboost import CatBoostRegressor, Pool

# Initialize CatBoostRegressor for multivariate regression
cat = CatBoostRegressor(loss_function='MultiRMSE', iterations=100, depth=15, learning_rate=0.0333)

# Convert training data to CatBoost Pool
train_pool = Pool(x_train, label=y_train)

# Train the model
cat.fit(train_pool)

# Make predictions on the test set
y_pred = cat.predict(test_standard)

# Display predictions
print('Predictions:')
print(y_pred)


0:	learn: 1.1753796	total: 2.12s	remaining: 3m 29s
1:	learn: 1.1734892	total: 2.14s	remaining: 1m 44s
2:	learn: 1.1694062	total: 4.26s	remaining: 2m 17s
3:	learn: 1.1655357	total: 6.56s	remaining: 2m 37s
4:	learn: 1.1614189	total: 8.81s	remaining: 2m 47s
5:	learn: 1.1581193	total: 11.2s	remaining: 2m 55s
6:	learn: 1.1549561	total: 13.7s	remaining: 3m 1s
7:	learn: 1.1522711	total: 16s	remaining: 3m 4s
8:	learn: 1.1489938	total: 18.3s	remaining: 3m 4s
9:	learn: 1.1458034	total: 20.8s	remaining: 3m 7s
10:	learn: 1.1423415	total: 23.1s	remaining: 3m 6s
11:	learn: 1.1393505	total: 25.9s	remaining: 3m 9s
12:	learn: 1.1367135	total: 28.5s	remaining: 3m 10s
13:	learn: 1.1337771	total: 31.3s	remaining: 3m 12s
14:	learn: 1.1306375	total: 34.1s	remaining: 3m 13s
15:	learn: 1.1282973	total: 36.7s	remaining: 3m 12s
16:	learn: 1.1258878	total: 38.8s	remaining: 3m 9s
17:	learn: 1.1229734	total: 41.3s	remaining: 3m 7s
18:	learn: 1.1200944	total: 43.4s	remaining: 3m 4s
19:	learn: 1.1168685	total: 45.9s

사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1

In [17]:
res = []
for a,b,c,d in y_pred:
    res.append(np.round(a*10+b*5+c*3+d*1,5))
res

[4.43955,
 3.82358,
 6.1752,
 4.55505,
 4.78991,
 4.83939,
 5.18453,
 4.26463,
 4.84012,
 5.31174,
 4.93732,
 4.36966,
 4.78776,
 4.31851,
 3.62024,
 5.3931,
 5.09027,
 4.85677,
 4.79466,
 5.04698,
 4.6654,
 3.85924,
 4.9383,
 4.95337,
 4.45547,
 4.87261,
 4.25163,
 5.70711,
 4.16778,
 5.06105,
 5.31034,
 5.89562,
 4.91175,
 4.37634,
 4.17655,
 4.01134,
 4.2795,
 6.43907,
 5.91661,
 5.8895,
 4.03383,
 4.34005,
 4.55616,
 4.83982,
 4.41794,
 4.21347,
 4.58926,
 4.39619,
 4.70259,
 4.48105,
 4.47008,
 4.78253,
 3.97969,
 4.48814,
 4.40321,
 4.74967,
 4.222,
 3.7949,
 4.27491,
 3.93346,
 3.81958,
 5.69368,
 5.78719,
 4.61754,
 3.79149,
 4.81379,
 4.23101,
 5.87674,
 4.05832,
 4.29019,
 4.3926,
 5.33039,
 3.95349,
 5.31477,
 3.98581,
 4.68472,
 4.29963,
 4.77538,
 6.21852,
 3.8233,
 5.03815,
 4.6286,
 4.47723,
 4.79664,
 4.15209,
 5.21411,
 4.68148,
 5.10594,
 3.79295,
 4.15095,
 4.75274,
 4.86779,
 5.57729,
 4.61242,
 4.83823,
 4.76885,
 4.66837,
 4.30337,
 4.83344,
 4.68014,
 5.12729,
 4

In [18]:
submission = pd.read_csv('C:/Users/User/Desktop/대구교통사고예측/sample_submission.csv')
submission['ECLO'] = res
submission.to_csv('C:/Users/User/Desktop/대구교통사고예측/cat_eclo.csv',index=False)

In [19]:
submission

,ID,ECLO
0,ACCIDENT_39609,4.43955
1,ACCIDENT_39610,3.82358
2,ACCIDENT_39611,6.17520
3,ACCIDENT_39612,4.55505
4,ACCIDENT_39613,4.78991
...,...,...
10958,ACCIDENT_50567,4.90262
10959,ACCIDENT_50568,4.70556
10960,ACCIDENT_50569,5.23509
10961,ACCIDENT_50570,4.84320


ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1 
# 각자 예측해서 ECLO계산해보기